Задание 1: 

Вычислить $\sqrt[k]a$:
* 1б С помощью метода бисекции.
* 2б C помощью метода Ньютона

Бисекция:

In [1]:
EPS = 1e-10

def bisection(f, left, right):
    while right - left > EPS:
        mid = (left + right) / 2
        if f(left) == 0:
            return left
        if f(right) == 0:
            return right
        if f(mid) == 0:
            return mid
        
        if f(left) * f(mid) < 0:
            right = mid
        elif f(right) * f(mid) < 0:
            left = mid
            
    return left

def find_root_bisection(k, a):
    if not k % 2 and a < 0:
        return None
    
    def func(x):
        return x ** k - a
    
    left_boundary = 0 if a > 0 else a
    ans_root = bisection(func, left_boundary, a ** 2)
    return ans_root

TESTS = [(2, 4), (3, 5), (3, -1000), (2, -49), (4, 16), (5, -32), (2, 1.44)]
for test in TESTS:
    k, a = test
    print('k = {}, a = {}: root is {}'.format(k, a, find_root_bisection(k, a)))


k = 2, a = 4: root is 2.0
k = 3, a = 5: root is 1.709975946596387
k = 3, a = -1000: root is -10.00000000004797
k = 2, a = -49: root is None
k = 4, a = 16: root is 2.0
k = 5, a = -32: root is -2.000000000027285
k = 2, a = 1.44: root is 1.1999999999485904


Методом Ньютона:

In [2]:
ITERS = 1000

def find_root_newton(k, a):
    if not k % 2 and a < 0:
        return None
    
    x = 2
    for _ in range(ITERS):
        x = ((k - 1) * (x ** k) + a) / (k * (x ** (k - 1)))
    return x

for test in TESTS:
    k, a = test
    # malkovsky: рекомендуется использовать f-string вместо format
    print('k = {}, a = {}: root is {}'.format(k, a, find_root_newton(k, a)))

k = 2, a = 4: root is 2.0
k = 3, a = 5: root is 1.709975946676697
k = 3, a = -1000: root is -10.0
k = 2, a = -49: root is None
k = 4, a = 16: root is 2.0
k = 5, a = -32: root is -2.0
k = 2, a = 1.44: root is 1.2


Задание 2:

Дан многочлен P степени не больше 5 и отрезок $[L; R]$, пара:
* 2б Локализовать корни P, т.е. найти непересекающиеся отрезки $[L_1; R_1], . . . , [L_m; R_m]$
такие, что $P(L_i)P(R_i) < 0$.
* 1б Для каждого $[L_i; R_i]$ найти соответствующий корень.
* 2б Найти глобальный минимум P на отрезке $[L; R]$.


Локализация и нахождение корней:

Заметим, что на промежутках монотонности функции есть ровно по одному корню многочлена. Тогда нам достаточно найти нули производной и взять отрезки между ними (но!! нужно чтобы на концах отрезка были значения разных знаков). Производная многочлена степени n является многочленом (n - 1)-ой степени. Таким образом, мы свели задачу к меньшей (будем рекурсивно находить корни, пока не дойдём до многочлена 1-ой степени).

In [3]:
def poly_value(coef: list(), x):
    ans = 0
    curr_deg = 1
    for c in coef:
        ans += c * curr_deg
        curr_deg *= x
    return ans


def poly_derivative(coef: list()):
    new_coef = []
    for i in range(1, len(coef)):
        new_coef.append(coef[i] * i)
    return new_coef


def is_appr_root(coef: list(), x, eps=0.00000001):
    return abs(poly_value(coef, x)) < eps


def intervals_and_roots(coef: list()):
    assert(len(coef) >= 2 and len(coef) <= 6 and coef[-1])
    if len(coef) == 2:
        root = (-1) * coef[0] / coef[1]
        return ([[root - 1, root + 1]], [root])
    derivative = poly_derivative(coef)
    def_intervals, der_roots = intervals_and_roots(derivative)
    
    M = 0
    for i in range(len(derivative) - 1):
        M = max(M, abs(derivative[i]))
    M = M / abs(coef[-1])
    
    ans_intervals = []
    if not len(der_roots):
        ans_intervals.append([(-1) * (M + 1), M + 1])
    else:
        if poly_value(coef, (-1) * (M + 1)) * poly_value(coef, der_roots[0]) <= 0 or is_appr_root(coef, der_roots[0]):
            ans_intervals.append([(-1) * (M + 1), der_roots[0]])
        flag = False #чтобы не повторять отрезки с одинаковыми корнями, являющимися также корнями производной
        for i in range(1, len(der_roots)):
            if (is_appr_root(coef, der_roots[i]) or poly_value(coef, der_roots[i - 1]) * poly_value(coef, der_roots[i]) <= 0) and not flag:
                ans_intervals.append([der_roots[i - 1], der_roots[i]])
            flag = is_appr_root(coef, der_roots[i])
        if poly_value(coef, M + 1) * poly_value(coef, der_roots[-1]) <= 0 or is_appr_root(coef, der_roots[-1]):
            if len(der_roots) != 1 or poly_value(coef, der_roots[-1]) != 0:
                ans_intervals.append([der_roots[-1], M + 1])

    def func(x):
        return poly_value(coef, x)
    
    ans_roots = []
    for interval in ans_intervals:
        left, right = interval
        curr_root = bisection(func, left, right)
        ans_roots.append(curr_root)
    
    return (ans_intervals, ans_roots)

# x^2+2x+1; x^2+1; x^3+3x+1; x^4-6x^2-5; x^5-x^4+x^3-x^2+x-1; x^2+2x-3; x^4-3x^3+x^2+3x-2; 3x^2+x
POLY_TESTS = [[1, 2, 1], [1, 0, 1], [1, 3, 0, 1], [-5, 0, -6, 0, 1], 
              [-1, 1, -1, 1, -1, 1], [-3, 2, 1], [-2, 3, 1, -3, 1], [0, 1, 3]]

for test in POLY_TESTS:
    intervals, roots = intervals_and_roots(test)
    print('coefs are equal to {}. And intervals are {}. Roots are {}'.format(test, intervals, roots))

coefs are equal to [1, 2, 1]. And intervals are [[-3.0, -1.0]]. Roots are [-1.0]
coefs are equal to [1, 0, 1]. And intervals are []. Roots are []
coefs are equal to [1, 3, 0, 1]. And intervals are [[-4.0, 4.0]]. Roots are [-0.32218535465653986]
coefs are equal to [-5, 0, -6, 0, 1]. And intervals are [[-13.0, -1.732050807622727], [1.7320508075354155, 13.0]]. Roots are [-2.5964701783416766, 2.596470178261063]
coefs are equal to [-1, 1, -1, 1, -1, 1]. And intervals are [[-5.0, 5.0]]. Roots are [0.9999999999854481]
coefs are equal to [-3, 2, 1]. And intervals are [[-3.0, -1.0], [-1.0, 3.0]]. Roots are [-3.0, 1.0]
coefs are equal to [-2, 3, 1, -3, 1]. And intervals are [[-10.0, -0.4430004682430645], [-0.4430004682430645, 0.9999999999867832], [1.6930004681335005, 10.0]]. Roots are [-1.0000000000168396, 0.9999999946111876, 1.9999999999952467]
coefs are equal to [0, 1, 3]. And intervals are [[-1.3333333333333333, -0.16666666666666666], [-0.16666666666666666, 1.3333333333333333]]. Roots are [-0

Нахождение глобального минимума:

Глобальный минимум может быть либо в точках, где производная равна 0, либо на концах отрезка.

In [4]:
def find_global_min(coef: list(), left=-20, right=20):
    derivative = poly_derivative(coef)
    def_intervals, der_roots = intervals_and_roots(derivative)
    
    ans = min(poly_value(coef, left), poly_value(coef, right))
    for root in der_roots:
        ans = min(ans, poly_value(coef, root))
    return ans

for test in POLY_TESTS:
    print('coefs are equal to {}. And global minimum is {}'.format(test, find_global_min(test)))

coefs are equal to [1, 2, 1]. And global minimum is 0.0
coefs are equal to [1, 0, 1]. And global minimum is 1.0
coefs are equal to [1, 3, 0, 1]. And global minimum is -8059
coefs are equal to [-5, 0, -6, 0, 1]. And global minimum is -14.0
coefs are equal to [-1, 1, -1, 1, -1, 1]. And global minimum is -3368421
coefs are equal to [-3, 2, 1]. And global minimum is -4.0
coefs are equal to [-2, 3, 1, -3, 1]. And global minimum is -2.833422409000351
coefs are equal to [0, 1, 3]. And global minimum is -0.08333333333333333


Задание 3:

При a, b, c > 0 найти минимум функции $e^{ax} + e^{−bx} + c(x − d)^2$:
* 1б С использованием метода бисекции.
* 2б C использованием метода Ньютона.
* 1б С использованием тернарного поиска.

In [5]:
import numpy as np

Методом бисекции:

In [6]:
def find_min_bisection(a, b, c, d):
    
    def f(x):
        return np.exp(a * x) + np.exp(-b * x) + c * ((x - d) ** 2)
    def derivative(x):
        return a * np.exp(a * x) - b * np.exp(-b * x) + 2 * c * (x - d)
    
    left_boundary = -1
    while derivative(left_boundary) > 0:
        left_boundary *= 2
    if derivative(left_boundary) == 0:
        return f(left_boundary)
    # malkovsky: вот честно говоря не уверен по поводу такой границы, но наверно ок
    right_boundary = b + d
    x_min = bisection(derivative, left_boundary, right_boundary)
    
    return f(x_min)

SOME_TESTS = [(1, 2, 3, 4), (3, 2, 1, 0), (1, 1, 1, 1)]
for test in SOME_TESTS:
    a, b, c, d = test
    print('a = {}, b = {}, c = {}, d = {} and minimum is equal to {}'.format(a, b, c, d, find_min_bisection(a, b, c, d)))

a = 1, b = 2, c = 3, d = 4 and minimum is equal to 18.652352027148623
a = 3, b = 2, c = 1, d = 0 and minimum is equal to 1.965755262275268
a = 1, b = 1, c = 1, d = 1 and minimum is equal to 2.505042602807782


Методом Ньютона:

In [7]:
def find_min_newton(a, b, c, d):
    
    def f(x):
        return np.exp(a * x) + np.exp(-b * x) + c * ((x - d) ** 2)
    def derivative(x):
        return a * np.exp(a * x) - b * np.exp(-b * x) + 2 * c * (x - d)
    def dxdx(x):
        return (a ** 2) * np.exp(a * x) + (b ** 2) * np.exp(-b * x) + 2 * c
    
    x_min = 0
    for _ in range(ITERS): 
        x_new = x_min - (derivative(x_min) / dxdx(x_min))
        if abs(x_new - x_min) < EPS:
            break
        x_min = x_new
        
    return f(x_min)

for test in SOME_TESTS:
    a, b, c, d = test
    print('a = {}, b = {}, c = {}, d = {} and minimum is equal to {}'.format(a, b, c, d, find_min_newton(a, b, c, d)))

a = 1, b = 2, c = 3, d = 4 and minimum is equal to 18.652352027148623
a = 3, b = 2, c = 1, d = 0 and minimum is equal to 1.965755262275268
a = 1, b = 1, c = 1, d = 1 and minimum is equal to 2.505042602807782


С помощью тернарного поиска:

In [8]:
def find_min_ternary(a, b, c, d):
    
    def f(x):
        return np.exp(a * x) + np.exp(-b * x) + c * ((x - d) ** 2)
    def derivative(x):
        return a * np.exp(a * x) - b * np.exp(-b * x) + 2 * c * (x - d)
    
    left_boundary = -1
    while derivative(left_boundary) > 0:
        left_boundary *= 2
    if derivative(left_boundary) == 0:
        return f(left_boundary)
    right_boundary = b + d
    
    while right_boundary - left_boundary > EPS:
        m1 = left_boundary + (right_boundary - left_boundary) / 3
        m2 = right_boundary - (right_boundary - left_boundary) / 3
        if f(m1) < f(m2):
            right_boundary = m2
        else:
            left_boundary = m1
    return f(left_boundary)

for test in SOME_TESTS:
    a, b, c, d = test
    print('a = {}, b = {}, c = {}, d = {} and minimum is equal to {}'.format(a, b, c, d, find_min_ternary(a, b, c, d)))

a = 1, b = 2, c = 3, d = 4 and minimum is equal to 18.652352027148623
a = 3, b = 2, c = 1, d = 0 and minimum is equal to 1.9657552622752683
a = 1, b = 1, c = 1, d = 1 and minimum is equal to 2.505042602807782
